# PLAN
## Initial
#### Safe bet to use Alpha Vantage to develop Model
## Final Step
#### Once ready to do extensive tranning with Multiple Stocks we will need to use SEC EDGAR most likely (need to figure that out) or pay for like a month subscription of Alpha Vantage

In [95]:
import requests
import pandas as pd
from datetime import datetime, timedelta


# Using FMP
# - This is more accurate and consitent! Yahoo is web scrapping
# - Provides the financial statetement for a broader range then yahoo finance

# Key which is given from my FMP account. (This key is needed to make requests) Maximum of 200 calls/day
API_KEY = "wUvM2M29ZVxHDvK8IRp2P7iyrT8uhQG4"

ticker = "AAPL"

end_date = datetime(2024, 6, 1)
start_date = end_date - timedelta(days=120)

url_hist_stock_price = f"https://financialmodelingprep.com/stable/historical-price-eod/light?symbol={ticker}&from={start_date.date()}&to={end_date.date()}&apikey={API_KEY}"

# Historical Data Request
response_hist_stock_price = requests.get(url_hist_stock_price)

json_response_hist_stock_price = response_hist_stock_price.json()


In [96]:
closing_price = pd.DataFrame(json_response_hist_stock_price)

# Convert 'date' to datetime type
closing_price['date'] = pd.to_datetime(closing_price['date'])

# Keep only 'date' and 'price' columns
closing_price = closing_price[['date', 'price']]

closing_price['price'] = pd.to_numeric(closing_price['price'], errors='coerce')
closing_price['date'] = pd.to_datetime(closing_price['date'], errors='coerce')
# Set 'date' as the index
closing_price.set_index('date', inplace=True)

print(closing_price)

             price
date              
2024-05-31  192.25
2024-05-30  191.29
2024-05-29  190.29
2024-05-28  189.99
2024-05-24  189.98
...            ...
2024-02-08  188.32
2024-02-07  189.41
2024-02-06  189.30
2024-02-05  187.68
2024-02-02  185.85

[83 rows x 1 columns]


In [97]:
# ALPHA VANTAGE EXPERIMENTING (https://www.alphavantage.co/documentation/#)
# gives some insight into the different attributes without sending demo request: https://documentation.alphavantage.co/FundamentalDataDocs/gaap_documentation.html

API_KEY = "FJLURMBXQEXWW9PP"

ticker = "AAPL"

def getInfo(function:str,ticker:str):
    url = f"https://www.alphavantage.co/query?function={function.upper()}&symbol={ticker}&apikey={API_KEY}"
    response = requests.get(url)
    return response.json()

In [98]:
# Requests
r_income_statements = getInfo("income_statement",ticker)
r_balance_sheet = getInfo("balance_sheet",ticker)
r_cash_flow = getInfo("cash_flow",ticker)

In [99]:
# Dataframes
df_income_statement = pd.DataFrame(r_income_statements["quarterlyReports"])
df_income_statement['fiscalDateEnding'] = pd.to_datetime(df_income_statement['fiscalDateEnding'])

#df_income_statement = df_income_statement.rename(columns={"fiscalDateEnding": "dates"})
df_income_statement.set_index("fiscalDateEnding", inplace=True)
df_income_statement = df_income_statement.apply(pd.to_numeric, errors='coerce')
df_income_statement = df_income_statement.fillna(0)



df_balance_sheet = pd.DataFrame(r_balance_sheet["quarterlyReports"])
df_balance_sheet['fiscalDateEnding'] = pd.to_datetime(df_balance_sheet['fiscalDateEnding'])
#df_balance_sheet = df_balance_sheet.rename(columns={"fiscalDateEnding": "dates"})
df_balance_sheet.set_index("fiscalDateEnding", inplace=True)
df_balance_sheet = df_balance_sheet.apply(pd.to_numeric, errors='coerce')
df_balance_sheet = df_balance_sheet.fillna(0)


df_cash_flow = pd.DataFrame(r_cash_flow["quarterlyReports"])
df_cash_flow['fiscalDateEnding'] = pd.to_datetime(df_cash_flow['fiscalDateEnding'])
#df_cash_flow = df_cash_flow.rename(columns={"fiscalDateEnding": "dates"})
df_cash_flow.set_index("fiscalDateEnding", inplace=True)
df_cash_flow = df_cash_flow.apply(pd.to_numeric, errors='coerce')
df_cash_flow = df_cash_flow.fillna(0)


In [55]:
# Available data
#print("Income Statement\n",df_income_statement.columns)
#print("Balance Sheet\n",df_balance_sheet.columns)
#print("Cash Flow\n",df_cash_flow.columns)

In [100]:
# TODO: Adjust all metrics such that they can be mapped to that of closing price (i.e the dates)

# Often used terms
net_income = df_income_statement['netIncome']
equity = df_balance_sheet["totalShareholderEquity"]
outstandingShares = df_balance_sheet["commonStockSharesOutstanding"]
book_value = equity
book_value_per_share = book_value/outstandingShares
total_revenue =df_income_statement['totalRevenue']

# Profitability Metrics
EPS = (net_income-df_cash_flow["dividendPayoutPreferredStock"])/outstandingShares
ROE = net_income/equity
net_Profit_Margin = net_income/total_revenue

# Valuation Metrics--> MOST PROBLEMATIC HERE
#P_E_Ratio = closing_price/EPS
#P_B_Ratio = closing_price/book_value_per_share
#P_S_Ratio = closing_price*outstandingShares/total_revenue

# Risk and Efficiency metrics
D_E_Ratio = df_balance_sheet["shortLongTermDebtTotal"]/equity
Current_Ratio = df_balance_sheet["totalCurrentAssets"]/df_balance_sheet["totalCurrentLiabilities"]



In [ ]:
# INITIAL FEATURES

import pandas as pd
import numpy as np

columns_part1 = []
# Daily Return
daily_return = closing_price.pct_change()

# 5-Day Return
ret_5d = closing_price.pct_change(5)
# 10-Day Return
ret_10d = closing_price.pct_change(10)

# 5-Day Volatility (Std Dev of Returns)
vol_5d = closing_price.pct_change().rolling(window=5).std()

# 10-Day Volatility
vol_10d = closing_price.pct_change().rolling(window=10).std()

# Momentum (10d)
momentum_10d = closing_price - closing_price.shift(10)

# SMA_10/SMA_50 Ratio
sma_10 = closing_price.rolling(window=10).mean()
sma_50 = closing_price.rolling(window=50).mean()
sma_ratio = sma_10/sma_50

# Z-score (20d)
rolling_mean = closing_price.rolling(window=20).mean()
rolling_std = closing_price.rolling(window=20).std()
z_score_20d = (closing_price - rolling_mean)/rolling_std

# RSI (14d)
delta = closing_price.diff()
gain = delta.where(delta > 0, 0.0)
loss = -delta.where(delta < 0, 0.0)

avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain/avg_loss
rsi_14 = 100 - (100 / (1 + rs))

dataframes = [
    (closing_price,'price'),
    (daily_return, 'ret_1d'),
    (ret_5d, 'ret_5d'),
    (ret_10d, 'ret_10d'),
    (vol_5d, 'vol_5d'),
    (vol_10d, 'vol_10d'),
    (momentum_10d, 'momentum_10d'),
    (sma_ratio, 'sma_ratio_10_50'),
    (z_score_20d, 'zscore_20d'),
    (rsi_14, 'rsi_14')
    ()
]

df_features_short_term = pd.concat(
    [df.rename(columns={df.columns[0]: name}) for df, name in dataframes],
    axis=1
)


In [173]:

#print(df_balance_sheet.columns)

outstandingShares_str = df_balance_sheet["commonStockSharesOutstanding"]
outstandingShares = outstandingShares_str.apply(pd.to_numeric, errors='coerce')
df_metrics = pd.DataFrame()

df_features_long_term = pd.concat([
    EPS.rename('EPS'),
    ROE.rename('ROE'),
    D_E_Ratio.rename('D_E_Ratio'),
    Current_Ratio.rename('Current_Ratio')

], axis=1)


combined_rows = []
quarter_dates = df_features_long_term.index
pos = 0

columns = ['date']
columns.extend(df_features_short_term.columns.to_list())
columns.extend(df_features_long_term.columns.to_list())
columns.extend(['P_E_ratio','P_B_ratio','P_S_ratio'])

for date,row in df_features_short_term.iterrows():
    index = quarter_dates[pos]
    if (pos < len(quarter_dates)):
        if index < date:
            new_row = [date]
            new_row.extend(row.to_list())
            new_row.extend(df_features_long_term.loc[index].to_list())

            bv_per_share_elem = book_value_per_share.loc[index]
            outstandingShares_elem = outstandingShares.loc[index]
            total_revenue_elem = total_revenue.loc[index]
            
            P_E_ratio = row.price/df_features_long_term.loc[index]['EPS']
            P_B_ratio = row.price/bv_per_share_elem
            P_S_ratio = row.price*outstandingShares_elem/total_revenue_elem

            new_row.extend([P_E_ratio,P_B_ratio,P_S_ratio])

            combined_rows.append(new_row)
        else:
            pos+=1
df_features = pd.DataFrame(combined_rows,columns=columns)
df_features = df_features.dropna()
df_features.set_index('date',inplace=True)
print(df_features.head())


  # NEED TO COMBINE THE ROWS TOGHETHER INTO!



             price    ret_1d    ret_5d   ret_10d    vol_5d   vol_10d  \
date                                                                   
2024-03-21  171.37 -0.005282 -0.000641  0.010555  0.013119  0.009809   
2024-03-20  178.67  0.042598  0.030927  0.058346  0.023394  0.016110   
2024-03-19  176.08 -0.014496  0.037535  0.037902  0.021682  0.017350   
2024-03-18  173.72 -0.013403  0.016798  0.028903  0.023659  0.018029   
2024-03-15  172.62 -0.006332  0.001974  0.015233  0.023825  0.018191   

            momentum_10d  sma_ratio_10_50  zscore_20d     rsi_14       EPS  \
date                                                                         
2024-03-21          1.79         0.963099    0.343570  42.789784  2.177363   
2024-03-20          9.85         0.970146    2.589364  71.361318  2.177363   
2024-03-19          6.43         0.975458    1.510199  62.237495  2.177363   
2024-03-18          4.88         0.980061    0.659479  59.641420  2.177363   
2024-03-15          2.59   

In [ ]:
# Train
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Features df
X = df_features.drop(columns=['y'])

# Target Columns
y = df_features['y']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Train a Linear Regression model (least squared)
model = LinearRegression()
model.fit(X_train, y_train) # Find A in y=Ax

# Predict and evaluate
y_pred = model.predict(X_test) # Predict y_test using the A matrix found and X_test
mse = mean_squared_error(y_test,y_pred) # Compare the found y_test (y_prep) to the true y_test
print('Mean Squared Error', mse)

print('Min of y:', y.min())
print('Max of y:', y.max())

range_y = y.max() - y.min()
print('range : ', range_y)

pour_error = ((mse**(1/2)/range_y)*100)
print('error :', pour_error, '%')

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
r2 = r2_score(y_test, y_pred)
rmse = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print(r2,rmse,mse)

q1 = y.quantile(0.25)
q3 = y.quantile(0.75)
iqr = q3 - q1
nRMSE = (mse*(1/2)) / iqr
print(nRMSE) # <0.5 excellent, 0.5-1.0 good, 1.0-1.5 moderate, >1.5 Weak